<a href="https://colab.research.google.com/github/yashboura303/House_Prediction/blob/master/House_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Richer syntax highlighting

Improved support for nested languages:

```notebook-python
df = pd.io.gbq.read_gbq('''
  SELECT 
    REGEXP_EXTRACT(name, '[a-zA-Z]+'),
    SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''')
```

In [5]:
from bs4 import BeautifulSoup
import requests
import csv
l = []
for i in range(60):
	source = requests.get(f'https://www.magicbricks.com/flats-in-mumbai-for-sale-pppfs/page-{i}').text
	soup = BeautifulSoup(source, 'lxml')
	for div in soup.find_all("div", class_='flex relative clearfix m-srp-card__container'):
		l.append(div)
		# print("Price", end=" ")
		# print(div.find("div", class_="m-srp-card__price").text)
		for div in (div.find_all("div", class_="m-srp-card__summary__item")):
			# print((div.find("div", class_="m-srp-card__summary__title").text), end=" ")
			# print(div.find("div", class_="m-srp-card__summary__info").text)
	print("*"*50)
print("Count:", len(l))

IndentationError: ignored